# Split Subjective Days

In [364]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import seaborn as sns
import pylab as py
import statistics

# Summary Filename

In [ ]:
# Read Spider FRP CSV file
summary_filename = 'Metazygia wittfeldae summary'
summary_df = pd.read_csv(frp_filename + '.csv')

# Get FRP Value of Specific Spider

In [351]:
# Extract FRP values for each spider and create new CSV file

def get_frp(summary_df, spider_name):
    # Creating new FRP CSV file from summary CSV
    summary_df = summary_df[(summary_df['Conditions'] == 'DD') & (summary_df['Spider ID'].str.match('Monitor 1'))]
    frp_df = summary_df[['Spider ID', 'LombSc period']]
    frp_df = frp_df.reset_index(drop = True)
    frp_df = frp_df.set_index('Spider ID')
    frp_df.to_csv(summary_filename + '_period')
    
    # Get FRP value of specific spider
    frp = float(frp_df.values[frp_df.index.str.endswith(spider_name)])
    
    return frp

get_frp(summary_df, 'Monitor 1 Spider 1')

22.0111503

# Activity Filename

In [278]:
# Read spider activity CSV file

filename = 'Metazygia wittfeldae Monitor 1 Updated_DD_binary'
df = pd.read_csv(filename + '.csv', index_col = 0)
df.index = pd.to_datetime(df.index)
#display(df)

# Split Subjective Days Function

Given arguments dataframe in DateTimeIndex, the initial day where the lights didn't turn on, and the FRP of the spider, returns the subjective days of the spider.

In [354]:
def split_subjective_days(df, initial_day, frp):
    # Get day where lights would turn on but they don't (DD)
    df_start = df.at_time('07:00:00')
    idx = df_start.index[0]
    
    # Subjective days starting from lights not being turned on to given FRP
    dti = pd.date_range(idx, periods = 10, freq = frp)
    dti = dti.round(freq = '1T')
    
    return dti

split_subjective_days(df, idx, str(frp) + 'H')

DatetimeIndex(['2017-04-26 07:00:00', '2017-04-27 05:01:00',
               '2017-04-28 03:01:00', '2017-04-29 01:02:00',
               '2017-04-29 23:03:00', '2017-04-30 21:03:00',
               '2017-05-01 19:04:00', '2017-05-02 17:05:00',
               '2017-05-03 15:05:00', '2017-05-04 13:06:00'],
              dtype='datetime64[ns]', freq=None)

# Extracting Activity

In [419]:
def extract_activity_from_subjective_day(dti, start_index, end_index, column_name):
    # Extract activity based on subjective day index
    activitydf = df.loc[dti[start_index] : dti[end_index], [column_name]]
    
    #activitydf.to_csv(filename + '_subjective.csv')
    
    return activitydf

extract_activity_from_subjective_day(dti, 3, 4, 'Monitor 1 Spider 1')

,Monitor 1 Spider 1
Date-Time,
2017-04-29 01:02:00,0
2017-04-29 01:03:00,0
2017-04-29 01:04:00,0
2017-04-29 01:05:00,0
2017-04-29 01:06:00,0
...,...
2017-04-29 22:59:00,0
2017-04-29 23:00:00,0
2017-04-29 23:01:00,0


In [418]:
def probability_in_sections(activitydf, start_index, end_index):
    list_of_dates = activitydf.loc[dti[start_index] : dti[end_index]].index.tolist()
    median_date = statistics.median(list_of_dates)
    display(median_date)
    
    sub_day = activitydf.loc[dti[start_index] : median_date]
    rows = len(sub_day.index)
    total_day_movement = sub_day.sum()
    prob_day = total_day_movement / rows
    
    sub_night = activitydf.loc[median_date : dti[end_index]]
    rows = len(sub_night.index)
    total_night_movement = sub_night.sum()
    prob_night = total_night_movement / rows
    
    return prob_day, prob_night

probability_in_sections(activitydf, 1, 2)

Timestamp('2017-04-27 16:01:00')

(Monitor 1 Spider 1    0.027231
 dtype: float64,
 Monitor 1 Spider 1    0.23298
 dtype: float64)

In [431]:
list_of_dates = activitydf.loc[dti[1] : dti[2]].index.tolist()
#print(list_of_dates)
dates = extract_activity_from_subjective_day(dti, 3, 4, 'Monitor 1 Spider 1').index.tolist()
#print(dates)
#median_date = statistics.median(list_of_dates)
median_dates = statistics.median(dates)
#display(median_date)
display(median_dates)

TypeError: unsupported operand type(s) for +: 'Timestamp' and 'Timestamp'